# Photo and Video

In [1]:
# очистка заднего фона на изображении перед выполнением сравнения
from src.clear import clear_image_background

file = "./original_photo/1.jpg"
clear_image_background(file)


image 1/1 /home/monk/Practicum/Similarity/original_photo/1.jpg: 480x640 1 person, 1 handbag, 1 tie, 1 chair, 54.0ms
Speed: 1.1ms preprocess, 54.0ms inference, 41.9ms postprocess per image at shape (1, 3, 480, 640)


In [2]:
# простая проверка cosine_similarity между двумя изображениями

import cv2

from src.similarity import similarity

image_1 = cv2.imread("./masked_photo/1_mask_0.png")
image_2 = cv2.imread("./original_photo/1.jpg")

similarity(image_1, image_2)

/home/monk/Practicum/Similarity/.venv/lib/python3.10/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


0.7734463810920715

# Text

In [2]:
#

# Labels
# 0: NEUTRAL
# 1: POSITIVE
# 2: NEGATIVE


import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("blanchefort/rubert-base-cased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained(
    "blanchefort/rubert-base-cased-sentiment", return_dict=True
)


@torch.no_grad()
def predict(text):
    inputs = tokenizer(
        text, max_length=512, padding=True, truncation=True, return_tensors="pt"
    )
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    # predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted


text = "Я ненавижу твою прекрасную обворожительную, чудесную, фантастическую любовь"
predict(text)  # probability of labels

tensor([[0.1807, 0.0679, 0.7515]])

In [3]:
from transformers import AutoTokenizer, AutoModel
import torch


def text_similarity(
    query: str,
    description: str,
    model_ckpt: str = "sentence-transformers/all-mpnet-base-v2",
):
    """_summary_

    Args:
        query (str): _description_
        description (str): _description_
        model_ckpt (str, optional): _description_. Defaults to "sentence-transformers/all-mpnet-base-v2".
    """

    def split_sentinces(text: str) -> list[str]:
        result = []
        for line in text.strip().replace("\n", "").split("."):
            if line:
                result.append(line.strip())
        return result

    def mean_pooling(model_output, attention_mask):
        token_embeddings = model_output[
            0
        ]  # First element of model_output contains all token embeddings
        input_mask_expanded = (
            attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        )
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        return sum_embeddings / sum_mask

    # Encode text
    def encode(texts: list[str] | str, tokenizer: AutoTokenizer, model: AutoModel):

        # Tokenize sentences
        encoded_input = tokenizer(
            texts, max_length=512, padding=True, truncation=True, return_tensors="pt"
        )
        # Compute token embeddings
        with torch.no_grad():
            model_output = model(**encoded_input, return_dict=True)

        sentence_embeddings = mean_pooling(
            model_output, encoded_input["attention_mask"]
        )
        return sentence_embeddings

    results = {}

    docs = split_sentinces(description)
    queries = split_sentinces(query)

    tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
    model = AutoModel.from_pretrained(model_ckpt)

    # Encode docs
    doc_emb = encode(docs, tokenizer, model)

    for query in queries:
        # Encode every query
        query_emb = encode(query, tokenizer, model)

        # Compute dot score between query and all document embeddings
        scores = torch.mm(query_emb, doc_emb.transpose(0, 1))[0].cpu().tolist()

        results[query] = []
        for doc, score in zip(scores, docs):
            results[query].append((doc, score))

        # Sort by decreasing score for each query
        results[query] = sorted(results[query], key=lambda x: x[0], reverse=True)

    return results


query = """В видео должно рассказываться о новых кроссовках Nikeль модели Zoom x3000. В положительном ключе должно быть рассказано о характеристиках или общее впечатление."""

description = """
Представляем вам новую модель обуви от всемирно известного бренда Nikeль - Zoom x3000. Это идеальное сочетание инновационного дизайна, функциональности и комфорта, созданное для людей, стремящихся к величию в любых ситуациях.
Nikeль Zoom x3000 - это прорыв в мире спортивной обуви. Она оснащена уникальной системой амортизации Zoom Air, которая обеспечивает непревзойденную отдачу энергии при каждом шаге. Мягкий, но прочный верх из синтетического материала обеспечивает идеальную посадку, а специальные вставки для вентиляции помогают поддерживать ноги сухими и свежими.
Дизайн Zoom x3000 привлекает внимание своей смелостью и изяществом. Яркие цвета и современные линии создают эффект движения даже в состоянии покоя. Логотип Nikeль, изящно вписанный в общее оформление, дополняет образ, подчеркивая амбиции и стремление к победе его владельца.

Nikeль Zoom x3000 - это не просто обувь, это символ амбиций, стремления к совершенству и отказ от компромиссов. Это обувь для тех, кто не боится быть на виду и стремится быть на шаг впереди. В Nike Zoom x3000 вы всегда будете на вершине, независимо от того, где вы находитесь."""


text_similarity(
    query, greebok, model_ckpt="AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru"
)


# model_ckpt="blanchefort/rubert-base-cased-sentiment"

tokenizer_config.json:   0%|          | 0.00/516 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/781 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Some weights of XLMRobertaModel were not initialized from the model checkpoint at AlexKay/xlm-roberta-large-qa-multilingual-finedtuned-ru and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'В видео должно рассказываться о новых кроссовках Nikeль модели Zoom x3000': [(935.378662109375,
   'С GReebok C1500 вы всегда будете на шаг впереди'),
  (932.9957275390625,
   'Это воплощение инноваций и стиля, обувь, которая переопределяет привычные представления о комфорте и функциональности'),
  (930.1180419921875, 'Дизайн C1500 воплощает современные тенденции моды'),
  (910.0886840820312,
   'Верх из дышащего материала гарантирует оптимальную вентиляцию, а уникальная технология амортизации позволяет вашим ногам ощутить невероятную легкость при каждом шаге'),
  (905.8414916992188, 'GReebok - выбор победителей'),
  (820.012451171875,
   'Особое внимание уделяется деталям - эргономичная стелька обеспечивает дополнительный комфорт, а резиновые вставки на подошве предотвращают скольжение, делая ходьбу безопасной даже на скользкой поверхности'),
  (810.8298950195312,
   'Сочетание классической черной подошвы и ярко-красного верха делает эту обувь идеальным дополнением к любому образу, 

In [1]:
greebok = """
Открывая новые горизонты в мире обуви, наша вымышленная марка GReebok представляет вам свою новейшую модель - C1500. Это воплощение инноваций и стиля, обувь, которая переопределяет привычные представления о комфорте и функциональности.

Кроссовки C1500 выполнены из премиальных материалов, обеспечивая максимальную прочность и долговечность. Верх из дышащего материала гарантирует оптимальную вентиляцию, а уникальная технология амортизации позволяет вашим ногам ощутить невероятную легкость при каждом шаге.

Дизайн C1500 воплощает современные тенденции моды. Сочетание классической черной подошвы и ярко-красного верха делает эту обувь идеальным дополнением к любому образу, будь то повседневный городской стиль или спортивный аутфит.

Особое внимание уделяется деталям - эргономичная стелька обеспечивает дополнительный комфорт, а резиновые вставки на подошве предотвращают скольжение, делая ходьбу безопасной даже на скользкой поверхности.

С GReebok C1500 вы всегда будете на шаг впереди. Эта обувь - не просто аксессуар, это инвестиция в ваш комфорт, стиль и уверенность в себе. GReebok - выбор победителей.

"""